In [13]:
import pandas as pd
import pymssql
from clickhouse_driver import Client
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [14]:
def execute_clickhouse(query="select 'Test' as Test",
                       database='dns_retail',
                       external_tables=None,
                       **kwargs) -> pd.DataFrame:
    """
    Функция для выполнения запросов к Clickhouse

    **Аргументы:**

    - `query: str` - запрос
    - `database: str` - база данных (по-умолчанию `'dns_log'`)
    - `external_tables: pandas.DataFrame` - датафрэйм из которого бдует генерироваться внешняя таблица

    **Возвращает:**
    >    `pandas.DataFrame` с результатом запроса
    """
    client = Client(
        host='',
        port='',
        database=database,
        user='',
        password=''
    )
    
    if external_tables:
        pass
        #external_tables = [df_to_table(table[0], table[1]) for table in external_tables]

    rv = client.execute(query, external_tables=external_tables, with_column_types=True)
    df = pd.DataFrame(rv[0])
    df.columns = [i[0] for i in rv[1]]
    return df

def execute_pg_hr(query):
    """Функция обращения к хранилищу mart_hr DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""

    host = ''
    schema = ''
    login = ''
    password = ''

    conection_url = f'postgresql+psycopg2://{login}:{password}@{host}/{schema}'

    engine = create_engine(conection_url)
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

def get_postgres_connection_string(connection_id) -> str:
    """
    Возвращает строку подключения к postgres для указанного подключения
    :return:
    """
    class connection:
        login = ''
        password = ''
        host = ''
        port = ''
        schema = ''

    return f'postgresql+psycopg2://{connection.login}:{connection.password}@{connection.host}:{connection.port}/{connection.schema}'

def execute_onec(query):
    """Функция обращения к Хранилищу DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""
    ADuser = 'O'
    ADpassword = ''
    # ADuser = Config.ONEC_LOGIN
    # ADpassword = Config.ONEC_PASSWORD
    conn = pymssql.connect(server='', user=ADuser, password=ADpassword, database='')
    df = pd.read_sql(query, conn)
    
    return df



In [15]:
def GetPosLvlSalary():
    query = f"""                                                                                                                                                                               
	WITH first_query AS
	(
	SELECT  
	--		reg_table.Должность AS Должность
			dwh.[Справочник.ШтатноеРасписание].Наименование AS Должность
			,reg_table.Уровень AS Уровень
			,dwh.[Справочник.Города].Наименование AS Город
			,reg_table.Значение AS Зарплата
			, MAX(convert(date, Период)) OVER (PARTITION BY Должность, Уровень, Город) AS max_period
			,convert(date, reg_table.Период) as Период 
	FROM dwh.[РегистрСведений.УровеньЗаработнойПлаты] AS reg_table
	JOIN dwh.[Справочник.Города] ON reg_table.Город = dwh.[Справочник.Города].Ссылка 
	JOIN dwh.[Справочник.ШтатноеРасписание] ON reg_table.Должность  =  dwh.[Справочник.ШтатноеРасписание].Ссылка  
	)

	SELECT Город, Должность, Уровень, Зарплата, Период, CONVERT(DATE, GETDATE()) AS ДатаОбновления
	FROM  first_query
	WHERE Период = max_period
		AND Должность != '2.03. Фед.развитие сайта'
		AND Должность not like '%(на аутсорсе)%' 
	ORDER BY Зарплата """
    df = execute_onec(query)
    return df

In [16]:
def getActualStaff():
    query = f"""
  select
      as2."ДатаОбновления" 
    , as2."Должность"
    , as2."УровеньОплаты"
    , as2."ТекущийФилиал"
    ,count(*) as КоличествоСотрудников
  from actual_staff as2
  where as2."Должность" not like '%%(на аутсорсе)%%'
  group by as2."Должность", as2."РасчетнаяЗарплата", as2."УровеньОплаты", as2."ТекущийФилиал", as2."ДатаОбновления" 
  order by as2."Должность", as2."ТекущийФилиал", as2."УровеньОплаты" """
    df = execute_pg_hr(query)
    return df

In [17]:
def getBranchCity():
	query = f"""
	select 
	  bh.BranchCode 
	, bh.BranchName as "Филиал"
	, replace(bh.CityName, 'РКЦ ', '') as "Город"
from dns_retail.BranchHier bh 
	"""
	df = execute_clickhouse(query)
	return df

In [18]:
df_PosLvlSalary = GetPosLvlSalary()
df_PosLvlSalary

,Город,Должность,Уровень,Зарплата,Период,ДатаОбновления
0,Всеволожск,Управляющий СЦ В-класса,1.0,0.0,2020-06-01,2022-07-08
1,Великие Луки,Управляющий СЦ В-класса,1.0,0.0,2020-06-01,2022-07-08
2,Гатчина,Управляющий СЦ В-класса,1.0,0.0,2020-06-01,2022-07-08
3,Колпино,Управляющий СЦ В-класса,1.0,0.0,2020-06-01,2022-07-08
4,Кингисепп,Управляющий СЦ В-класса,1.0,0.0,2020-06-01,2022-07-08
...,...,...,...,...,...,...
63677,Санкт-Петербург,Директор дивизиона,3.0,300000.0,2021-12-01,2022-07-08
63678,Воронеж,Директор дивизиона,3.0,300000.0,2021-12-01,2022-07-08
63679,Владивосток,Директор,3.0,300000.0,2021-12-01,2022-07-08
63680,Владивосток,Федеральный директор,3.0,300000.0,2021-12-01,2022-07-08


In [19]:
df_actual_staff = getActualStaff()
df_actual_staff

,ДатаОбновления,Должность,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников
0,2022-07-08 14:36:46.910,Администратор киностудии,2.0,2421.0,1
1,2022-07-08 14:36:46.910,Администратор офиса,2.0,284.0,1
2,2022-07-08 14:36:46.910,Администратор офиса,3.0,292.0,1
3,2022-07-08 14:36:46.910,Администратор офиса,1.0,897.0,1
4,2022-07-08 14:36:46.910,Администратор офиса,2.0,897.0,1
...,...,...,...,...,...
18517,2022-07-08 14:36:46.910,Юрист,3.0,5822.0,1
18518,2022-07-08 14:36:46.910,Юрист,1.0,5823.0,2
18519,2022-07-08 14:36:46.910,Юрист,2.0,5823.0,1
18520,2022-07-08 14:36:46.910,Юрист,3.0,5823.0,1


In [20]:
df_branch_city = getBranchCity()
df_branch_city

,BranchCode,Филиал,Город
0,4652.0,Волоколамск ТЦ Кооператор,Волоколамск
1,1520.0,Северск на Коммунистическом (ЗАКРЫТО),Северск
2,1907.0,Самара ТЦ Юг ТП (ЗАКРЫТО),Самара
3,6103.0,"IT-группа 1С ""Учет и Рибейты""",Владивосток
4,2570.0,Тольятти ТЦ Вега (ЗАКРЫТО),Тольятти
...,...,...,...
6164,25.0,"Хаб Склад (ЗАКРЫТ, НЕ ИСПОЛЬЗОВАТЬ)",Хабаровск
6165,4093.0,Хадыженск Первомайская,Хадыженск
6166,3336.0,Нефтекамск Березовское шоссе,Нефтекамск
6167,4586.0,Кушва на Строителей,Кушва


In [21]:
df_checkerboard_salary = df_actual_staff \
    .merge(df_branch_city, left_on='ТекущийФилиал', right_on='BranchCode') \
    .merge(df_PosLvlSalary, left_on=['Город', 'Должность', 'УровеньОплаты'], right_on=['Город', 'Должность', 'Уровень'], how='left')

# df_checkerboard_salary['ДатаОбновления'] = df_checkerboard_salary['ДатаОбновления'].dt.strftime('%Y-%m-%d')
df_checkerboard_salary = df_checkerboard_salary[['Город', 'Филиал', 'Должность', 
                        'УровеньОплаты', 'Зарплата','КоличествоСотрудников']]
df_checkerboard_salary 
# 18563 47433 48650 18809

,Город,Филиал,Должность,УровеньОплаты,Зарплата,КоличествоСотрудников
0,Владивосток,Видеостудия Владивосток,Администратор киностудии,2.0,80000.0,1
1,Владивосток,Видеостудия Владивосток,Ведущий менеджер контента,1.0,70000.0,1
2,Владивосток,Видеостудия Владивосток,Главный редактор,2.0,80000.0,1
3,Владивосток,Видеостудия Владивосток,Руководитель киностудии,2.0,90000.0,1
4,Владивосток,Видеостудия Владивосток,Руководитель контентного направления,3.0,130000.0,1
...,...,...,...,...,...,...
18802,Владивосток,Фед. HR ФКО,Федеральный менеджер по подбору и работе с пер...,3.0,80000.0,1
18803,Владивосток,Фед. HR ФКО,Федеральный специалист коммерческих процессов,3.0,100000.0,1
18804,Санкт-Петербург,HR-Служба Спб,Федеральный менеджер по подбору и работе с пер...,3.0,80000.0,1
18805,Владивосток,Фед. отдел рекламных проектов (ЗАКРЫТО),Федеральный специалист по рекламе,2.0,50000.0,1


In [22]:
df_checkerboard_salary[df_checkerboard_salary['Зарплата'].isna()]['Город'].unique()


array(['Екатеринбург', 'Комсомольск-на-Амуре', 'Москва', 'Краснодар',
       'Пенза', 'Самара', 'Уфа', 'Челябинск', 'Красноярск', 'Владивосток',
       'Нижний Новгород', 'Благовещенск', 'Санкт-Петербург', 'Щёкино',
       'Вязьма', 'Ижевск', 'Казань', 'Иркутск', 'Новосибирск',
       'Евпатория'], dtype=object)

In [23]:
# filt = df_checkerboard_salary['Город'] == 'Владивосток' 
# df_checkerboard_salary.loc[filt][df_checkerboard_salary['Зарплата'].isna()]
qqq =df_checkerboard_salary[df_checkerboard_salary['Зарплата'].isna()]
qqq

,Город,Филиал,Должность,УровеньОплаты,Зарплата,КоличествоСотрудников
472,Екатеринбург,HR (д. Урал),Руководитель отдела по обучению и развитию пер...,2.0,NaN,1
880,Комсомольск-на-Амуре,Комсомольск Вокзальная ТП,Ведущий специалист в КО,NaN,NaN,1
1047,Москва,Москва Корпоративный отдел,Ведущий специалист в КО,NaN,NaN,1
1257,Краснодар,Краснодар Корпоратив,Ведущий специалист в КО,NaN,NaN,1
1275,Пенза,Пенза Корпоративный отдел,Ведущий специалист в КО,NaN,NaN,1
1768,Самара,Самара Склад,Кладовщик на РРЦ,NaN,NaN,1
1935,Уфа,Уфа Склад,Кладовщик на РРЦ,NaN,NaN,1
1936,Уфа,Уфа Склад,Кладовщик на РРЦ,NaN,NaN,2
2323,Челябинск,ЧБ РСЦ,Логист сервиса,NaN,NaN,1
2426,Красноярск,Красноярск Склад,Кладовщик на РРЦ,NaN,NaN,1
